<font size=5>

Classification with PySpark ML

Instruction, place data file "SMSSpamCollection" under data/SparkData folder before running the notebook

</font>

<font size=5> This dataset does not have column name, but we will give the proper columns.
</font>

In [1]:
!pip install findspark
!pip install matplotlib

import matplotlib as plt
%matplotlib inline
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
#Read Data
df = spark.read.csv("SMSSpamCollection (1).txt", sep = "\t", inferSchema=True, header = False)

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/hadoop/.local/lib/python3.9/site-packages
sysconfig: /home/hadoop/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None
Defaulting to user installation because normal site-packages is not writeable
distutils: /home/hadoop/.local/lib/python3.9/site-packages
sysconfig: /home/hadoop/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 22:19:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<font size=5> Show 3 lines to get an idea about the dataset,  _c0 looks like as a label, c1 looks feature </font>

In [2]:
spark.sparkContext.setLogLevel("error")

In [3]:
df.show(3, truncate = False)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0 |_c1                                                                                                                                                        |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|ham |Ok lar... Joking wif u oni...                                                                                                                              |
|spam|Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
+----+----------------

<font size=5> Note: Spam is Spam, Han is OK.  Rename Column name _c0 as status, _c1 as feature  </font>

In [4]:
df = df.withColumnRenamed('_c0', 'status').withColumnRenamed('_c1', 'message')
df.show(3, truncate = False)

+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|message                                                                                                                                                    |
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham   |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|ham   |Ok lar... Joking wif u oni...                                                                                                                              |
|spam  |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
+------+--

<font size=5>

Encode status column to numeric: ham to 1.0 and spam to 0. All our fields need to be numeric for machine to learn, also rename the column status to label
    
</font>

In [5]:
df.createOrReplaceTempView('temp')
df = spark.sql('select case status when "ham" then 1.0  else 0 end as label, message from temp')
df.show(3, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|message                                                                                                                                                    |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|1.0  |Ok lar... Joking wif u oni...                                                                                                                              |
|0.0  |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
+-----+---------

<font size=5> 1 is OK, 0 is Junk </font>

<font size=5>
Tokenize the messages
Tokenization is the process of taking text (such as a sentence) and breaking it into individual terms (usually words). Let’s tokenize the messages and create a list of words of each message.
</font>

In [6]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="message", outputCol="words")
wordsData = tokenizer.transform(df)
wordsData.show(3, False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|message                                                                                                                                                    |words                                                                                                                                                                                   |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------

<font size=5> CountVectorizer converts a collection of text documents to vectors of token counts.
    
See:
https://spark.apache.org/docs/latest/ml-features#countvectorizer


</font>


In [7]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show(3,False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|message                                                                                                                                                    |words                                                                                                                                                                                   |rawFeatures                                                                                         

<font size=5>
Apply Term frequency - inverse document frequency (TF-IDF)

#IDF reduces the features that often appear in the corpus. When using text as a feature, this usually improves performance because the most common, and therefore less important, words are weighted down.

</font>

In [8]:
from pyspark.ml.feature import  IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show(3,False)  #Only needed to train


+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [9]:
rescaledData.createOrReplaceTempView("rescaleData")

In [10]:
spark.sql("select features from rescaleData limit 5").show()


+--------------------+
|            features|
+--------------------+
|(13587,[8,42,52,6...|
|(13587,[5,75,411,...|
|(13587,[0,3,8,20,...|
|(13587,[5,22,60,1...|
|(13587,[0,1,66,87...|
+--------------------+



<font size=5>
Randomly Split DataFrame into 80% Training (trainDF) and 20 Testing (testDF)
    
</font>


In [11]:
seed = 0  # random seed 0
trainDF, testDF = rescaledData.randomSplit([0.8,0.2],seed)

<font size=5> counts of train and test DataFrame </font>

In [12]:
trainDF.count()

4424

In [13]:
testDF.count()

1150

<font size=5>
Try different classifiers.

Logistic regression classifier

Logistic regression is a common method of predicting classification responses. A special case of a generalized linear model is the probability of predicting a result. In spark.ml, logistic regression can be used to predict binary results by binomial logistic regression, or it can be used to predict multiple types of results by using multiple logistic regression. Use the family parameter to choose between these two algorithms, or leave it unset and Spark will infer the correct variable.

</font>

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
lr = LogisticRegression(maxIter = 100)

model_lr = lr.fit(trainDF)


In [15]:
prediction_lr = model_lr.transform(testDF)

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lr.evaluate(prediction_lr)

0.9475499400198194

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(prediction_lr)

0.9848942383213879

In [18]:
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(prediction_lr)

0.9852173913043478

In [19]:
train_fit_lr = prediction_lr.select('label','prediction')
train_fit_lr.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       1.0|   16|
|  1.0|       1.0|  995|
|  0.0|       0.0|  138|
|  1.0|       0.0|    1|
+-----+----------+-----+



<font size=5>
Naive Bayes
Naive Bayesian classifiers are a class of simple probability classifiers that apply strong (naive) independent assumptions between features based on Bayes' theorem. The spark.ml implementation currently supports polynomial naive Bayes and Bernoulli Naïve Bayes.
</font>

In [20]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()
Model_nb = nb.fit(trainDF)

In [21]:
predictions_nb = Model_nb.transform(testDF)
predictions_nb.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 5 rows



In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_nb = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_nb.evaluate(predictions_nb)

0.949864392635477

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_nb.evaluate(predictions_nb)

0.9372189798267707

In [24]:
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_nb.evaluate(predictions_nb)

0.9321739130434783

<font size=5>

Now let's try Random Forest Classfication to see how it performs on the classification on the same data
    
    
</font>

In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [26]:
rf = RandomForestClassifier(featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability', rawPredictionCol='rawPrediction',maxDepth=3)
Model_rf = rf.fit(trainDF)

In [27]:
predictions_rf = Model_rf.transform(testDF)
predictions_rf.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
+-----+----------+
only showing top 5 rows



In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_rf = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_rf.evaluate(predictions_rf)

0.5

<font size=5>

Given Area Under Curve is 0.5, we do not want to use this Random Forest Classification.  Area under Curve is between 0 to 1,
the more close to 1 the better the classication is.

We wil give up Random Forest on this classitication

</font>

In [29]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol='features', labelCol='label')

In [30]:
model_gbt=gbt.fit(trainDF)
predictions_gbt=model_gbt.transform(testDF)

In [31]:
predictions_gbt.show(5)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|             message|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0|2p per min to cal...|[2p, per, min, to...|(13587,[0,10,11,1...|(13587,[0,10,11,1...|[1.29041568799839...|[0.92961768434495...|       0.0|
|  0.0|3 FREE TAROT TEXT...|[3, free, tarot, ...|(13587,[0,10,11,5...|(13587,[0,10,11,5...|[0.74897381707673...|[0.81726817357518...|       0.0|
|  0.0|5 Free Top Polyph...|[5, free, top, po...|(13587,[0,3,15,34...|(13587,[0,3,15,34...|[1.70856338754911...|[0.96823552295241...|       0.0|
|  0.0|500 New Mobiles f...|[500, new, mobile...|(13587,[0,40,64,8...|(13587,[0,40,64,8...|[-0.5733911736343...|[0.24107729111878.

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_gbt = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_gbt.evaluate(predictions_gbt)

0.8768254837531946

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_gbt = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_gbt.evaluate(predictions_gbt)

0.9519440759890753

In [34]:
my_mc_gbt = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_gbt.evaluate(predictions_nb)

0.9321739130434783

<font size=5>

Model performance from Gradient Boosting Tree is good.

</font>

In [35]:
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(featuresCol="features", labelCol="label", maxIter=50)

In [36]:
model_lsvc=lsvc.fit(trainDF)
predictions_lsvc=model_lsvc.transform(testDF)

In [37]:
predictions_lsvc.show(5)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|             message|               words|         rawFeatures|            features|       rawPrediction|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0|2p per min to cal...|[2p, per, min, to...|(13587,[0,10,11,1...|(13587,[0,10,11,1...|[4.75723786234723...|       0.0|
|  0.0|3 FREE TAROT TEXT...|[3, free, tarot, ...|(13587,[0,10,11,5...|(13587,[0,10,11,5...|[4.00286362697667...|       0.0|
|  0.0|5 Free Top Polyph...|[5, free, top, po...|(13587,[0,3,15,34...|(13587,[0,3,15,34...|[4.58169856745822...|       0.0|
|  0.0|500 New Mobiles f...|[500, new, mobile...|(13587,[0,40,64,8...|(13587,[0,40,64,8...|[3.82684320293181...|       0.0|
|  0.0|500 New Mobiles f...|[500, new, mobile...|(13587,[0,40,64,8...|(13587,[0,40,64,8...|[3.82684320293181...|       0.0|
+-----+-

In [38]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lsvc = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lsvc.evaluate(predictions_gbt)

0.8768254837531946

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lsvc = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lsvc.evaluate(predictions_gbt)

0.9519440759890753

In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lsvc = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lsvc.evaluate(predictions_gbt)

0.9530434782608695

<font size=5>

Model performance from Linear Support Vector Machine is good.

</font>